#### 2-1 連立一次方程式を python の数理最適化ライブラリで解く
リンゴ120円、ナシ150円を合わせて10個買ったら1440円だった。それぞれ何個買ったのか？

In [23]:
from pulp import LpProblem, LpVariable, LpMaximize, LpStatus

問題の定義

In [24]:
apple_price = 120
pear_price = 150
total_price = 1440
total_quantity = 10

数理モデル、変数の定義

In [25]:
problem = LpProblem('SLE', LpMaximize)
x = LpVariable('x', cat='Continuous')
y = LpVariable('y', cat='Continuous')

条件式の定義

In [26]:
problem += apple_price * x + pear_price * y == total_price
problem += x + y == total_quantity

計算及び結果の表示

In [27]:
status = problem.solve()
print('Status=', LpStatus[status])
print('x=', x.value(), 'y=', y.value())

Status= Optimal
x= 2.0 y= 8.0


#### 2-2 線形計画問題を python の数理最適化ライブラリで解く
ある工場の利益を最大化する問題
- 製品 p を 1kg 製造するためには、原料 m が1kg、原料 n が2kg必要
- 製品 q を 1kg 製造するためには、原料 m が3kg、原料 n が1kg必要
- 原料 m の在庫は 30kg、原料 n の在庫は 40kg
- 単位量あたりの利益は、製品 p は1万円、製品 q は2万円

In [1]:
from pulp import LpProblem, LpVariable, LpMaximize, LpStatus

問題、数理モデル、変数の定義

In [2]:
mx, nx = 1, 2
my, ny = 3, 1
profit_x, profit_y = 10000, 20000

problem = LpProblem('LP', LpMaximize)
x = LpVariable('x', cat='Continuaus')
y = LpVariable('y', cat='Continuaus')

条件式の定義

In [3]:
problem += mx * x + my * y <= 30
problem += nx * x + ny * y <= 40
problem += x >= 0
problem += y >= 0
problem += profit_x * x + profit_y * y

計算及び結果表示

In [4]:
status = problem.solve()

print('Status:', LpStatus[status])
print('x=', x.value(), 'y=', y.value(), 'Obj=', problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 Obj= 260000.0


#### 2-3 規模の大きな数理最適化問題を Python のライブラリで解く
- ある工場の利益最大化問題
  - 製品 p1, p2, p3, p4 を製造
  - 原料 m1, m2, m3 の使用量ファイル(requires.csv)
  - 原料 m1, m2, m3 の在庫ファイル(stocks.csv)
  - 原料 m1, m2, m3 の利益ファイル(gains.csv)
- リスト
  - 製品 $[p_1, p_2, p_3, p_4]$
  - 原料 $[m_1, m_2, m_3]$
- 定数
  - 在庫 $stock_m$
  - 必要原料 $require_{p,m}$
  - 利益 $gain_p$
- 変数
  - 生産量 $x_p$
- 制約式
  - 生産数は 0 以上 $x_p \ge 0$
  - 在庫数の範囲 $\sum_P require_{p,m}\cdot x_p \le stock_m$
- 目的関数
  - 最大化 $Maximaize \sum_P gain_p \cdot x_p$

In [32]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus
import pandas as pd
from glob import glob

データ取得

In [33]:
requires_df = pd.read_csv(glob('../*/*/requires.csv')[0])
stocks_df = pd.read_csv(glob('../*/*/stocks.csv')[0])
gains_df = pd.read_csv(glob('../*/*/gains.csv')[0])
display(requires_df)
display(stocks_df)
display(gains_df)

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


,m,stock
0,m1,35
1,m2,22
2,m3,27


,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


リスト定義

In [34]:
products = gains_df['p'].to_list()
materials = stocks_df['m'].to_list()
products, materials

(['p1', 'p2', 'p3', 'p4'], ['m1', 'm2', 'm3'])

定数の定義

In [35]:
stock = stocks_df.set_index('m').to_dict()['stock']
require = {(row.p, row.m): row.require for row in requires_df.itertuples()}
gain = {row.p: row.gain for row in gains_df.itertuples()}
stock, gain

({'m1': 35, 'm2': 22, 'm3': 27}, {'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5})

In [36]:
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

数理モデル、変数の定義(線形計画問題と整数計画問題)

In [37]:
# problem = LpProblem('LP2', LpMaximize)
# x = LpVariable.dicts('x', products, cat='Continuous')
problem = LpProblem('IP', LpMaximize)
x = LpVariable.dicts('x', products, cat='Integer')

条件式、目的関数の定義

In [38]:
# 個別に記述した場合
# problem += x['p1'] >= 0
# problem += x['p2'] >= 0
# problem += x['p3'] >= 0
# problem += x['p4'] >= 0

for p in products:
    problem += x[p] >= 0

In [39]:
# 個別に記述
# m1_stock = [x['p1'] * require['p1', 'm1'], x['p2'] * require['p2', 'm1'], x['p3'] * require['p3', 'm1'], x['p4'] * require['p4', 'm1']]
# problem += lpSum(m1_stock) <= stock['m1']
# m2_stock = [x['p1'] * require['p1', 'm2'], x['p2'] * require['p2', 'm2'], x['p3'] * require['p3', 'm2'], x['p4'] * require['p4', 'm2']]
# problem += lpSum(m2_stock) <= stock['m2']
# m3_stock = [x['p1'] * require['p1', 'm3'], x['p2'] * require['p2', 'm3'], x['p3'] * require['p3', 'm3'], x['p4'] * require['p4', 'm3']]
# problem += lpSum(m3_stock) <= stock['m3']

# 材料ごとに記述
# problem += lpSum([require[p, 'm1'] * x[p] for p in products]) <= stock['m1']
# problem += lpSum([require[p, 'm2'] * x[p] for p in products]) <= stock['m2']
# problem += lpSum([require[p, 'm3'] * x[p] for p in products]) <= stock['m3']

for m in materials:
    problem += lpSum([require[p, m] * x[p] for p in products]) <= stock[m]

problem += lpSum([gain[p] * x[p] for p in products])

求解と結果の表示

In [40]:
status = problem.solve()
print('Status:', LpStatus[status])

for p in products:
    print(p, x[p].value())
print('Obj=', problem.objective.value())

Status: Optimal
p1 13.0
p2 3.0
p3 7.0
p4 -0.0
Obj= 79.0
